In [1]:
from algorithm.utils.data.tf.dataimage import DataImage
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import keras
%load_ext tensorboard

In [2]:
dataset = DataImage(data_path="Input/dataset", split=0.2, resize=True, height='auto', width='auto', normalize=True, mean='auto', name="Electronic components dataset", batch_size=16)

In [3]:
dataset

Electronic components dataset
	Number of points: 2884
	Root location: Input/dataset
	Split: 0.2
	Transform used:	Resize(size=(128, 128)
	Normalize(mean=[0.00392156862745098, 0.00392156862745098, 0.00392156862745098], std=None)


In [4]:
trainset = dataset.get_set(split="train")
validset = dataset.get_set(split="validation")

In [5]:
# get some random training images
dataiter = iter(trainset)
images, labels = next(dataiter)

In [6]:
images.shape, len(labels)

(TensorShape([16, 128, 128, 3]), 16)

In [7]:
images

<tf.Tensor: shape=(16, 128, 128, 3), dtype=float32, numpy=
array([[[[0.32625616, 0.58206195, 0.472258  ],
         [0.32625616, 0.58206195, 0.472258  ],
         [0.32625616, 0.58206195, 0.472258  ],
         ...,
         [0.3220895 , 0.5581495 , 0.4130515 ],
         [0.32778466, 0.5543012 , 0.41312474],
         [0.33215383, 0.5525276 , 0.41357234]],

        [[0.69393384, 0.91596204, 0.85984993],
         [0.69393384, 0.91596204, 0.85984993],
         [0.69393384, 0.91596204, 0.85984993],
         ...,
         [0.6182445 , 0.87633276, 0.78923106],
         [0.6269481 , 0.8771933 , 0.7912711 ],
         [0.6433364 , 0.88573843, 0.8037378 ]],

        [[0.99275434, 0.9934743 , 1.        ],
         [0.99275434, 0.9934743 , 1.        ],
         [0.99275434, 0.9934743 , 1.        ],
         ...,
         [0.96655947, 0.9967372 , 0.9986827 ],
         [0.97569084, 0.994392  , 0.9988679 ],
         [0.98748475, 1.        , 1.        ]],

        ...,

        [[0.6755822 , 0.9148285 ,

In [8]:
labels

<tf.Tensor: shape=(16,), dtype=string, numpy=
array([b'Resistor', b'Not mounted', b'Resistor', b'Resistor',
       b'Ceramic capacitor', b'Ceramic capacitor', b'Inductor',
       b'Resistor', b'Rectangular pin', b'Ceramic capacitor',
       b'Not mounted', b'Rectangular pin', b'Rectangular pin',
       b'Rectangular pin', b'Ceramic capacitor', b'Resistor'],
      dtype=object)>

In [9]:
class_names = dataset.labels

In [10]:
class_names

['Jumper',
 'Pushbutton',
 'Resistor Network',
 'Rectangular pin',
 'Relais',
 'Inductor',
 'Connector',
 'Optoisolator',
 'Fuse',
 'IC',
 'Minimelf',
 'Dip Switch',
 'LED',
 'Not mounted',
 'SOT',
 'Diode',
 'Ceramic capacitor',
 'Resistor',
 'Jumper cut',
 'TO252',
 'Oscillator',
 'Tantalum capacitor']

In [ ]:
import matplotlib.pyplot as plt
image_batch, label_batch = next(iter(trainset))

plt.figure(figsize=(60, 60))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].numpy().astype("uint8")) # to be implemented: normalization and resize
    label = label_batch[i].numpy().decode("utf-8")
    plt.title(label, fontsize=60)
    plt.axis("off")

In [12]:
from algorithm.deep.tf.neural_networks import ResNet2__1

In [13]:
trainset = dataset.apply_one_hot_encoding(trainset)
validset = dataset.apply_one_hot_encoding(validset)

In [14]:
out_dir = "out/models/ResNet2__1_2"

In [15]:
model = ResNet2__1(len(class_names),input_shape=(128, 128, 3))
tb_callback = keras.callbacks.TensorBoard(log_dir=f'{out_dir}/tensorboard_log', histogram_freq=1)

In [17]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"],
    run_eagerly=True,
)

In [18]:
model.fit(trainset, validation_data=validset, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
145/145 [==============================] - 55s 352ms/step - loss: 1.3627 - accuracy: 0.6200 - val_loss: 0.7092 - val_accuracy: 0.8125
Epoch 2/1000
145/145 [==============================] - 47s 320ms/step - loss: 0.5828 - accuracy: 0.8315 - val_loss: 0.5630 - val_accuracy: 0.8524
Epoch 3/1000
145/145 [==============================] - 46s 318ms/step - loss: 0.4031 - accuracy: 0.8800 - val_loss: 0.4689 - val_accuracy: 0.8819
Epoch 4/1000
145/145 [==============================] - 47s 323ms/step - loss: 0.2884 - accuracy: 0.9151 - val_loss: 0.4211 - val_accuracy: 0.8698
Epoch 5/1000
145/145 [==============================] - 45s 311ms/step - loss: 0.2252 - accuracy: 0.9311 - val_loss: 0.4187 - val_accuracy: 0.8750
Epoch 6/1000
145/145 [==============================] - 46s 313ms/step - loss: 0.2103 - accuracy: 0.9346 - val_loss: 0.3166 - val_accuracy: 0.9167
Epoch 7/1000
145/145 [==============================] - 46s 317ms/step - loss: 0.2166 - accuracy: 0.9294 - val_loss: 0

In [19]:
model.summary()

Model: "res_net2__1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 4, 4, 2048)        58370944  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 1000)              32769000  
                                                                 
 dense_1 (Dense)             (None, 22)                22022     
                                                                 
 model (Functional)          (None, 22)                91161966  
                                                                 
Total params: 91161966 (347.76 MB)
Trainable params: 32791022 (125.09 MB)
Non-trainable params: 58370944 (222.67 MB)
_________________________________________________________________


In [20]:
model.save(out_dir)

INFO:tensorflow:Assets written to: out/models/ResNet2__1_2/assets


INFO:tensorflow:Assets written to: out/models/ResNet2__1_2/assets


In [ ]:
keras.utils.plot_model(model, f'{out_dir}/ResNet2__1_2.svg', show_shapes=True, show_layer_activations=True, show_layer_names=True)

In [22]:
# Evaluation on unseen data.

test_loss, test_acc = model.evaluate(validset, verbose=2)
print('\nTest accuracy:', test_acc)

36/36 - 5s - loss: 1.1453 - accuracy: 0.9323 - 5s/epoch - 135ms/step

Test accuracy: 0.9322916865348816
